In [31]:
%matplotlib inline
import pandas as pd
from tsmom_model import *
import numpy as np

from datetime import datetime
import PPCA_MR3 as p
from linearmodels.asset_pricing import LinearFactorModel
import statsmodels.api as sm

In [32]:
import pylab
sns.set(font_scale=2)
pylab.rcParams['figure.figsize'] = (24, 18)

# Hand cleansed
Hand cleaned to removed duplicates

In [33]:
cleansed=load_amihud_markets_price()

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:158: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


# Amihud methodology

Amihud (2002) developed the illiquidity measure expressed by:

$$ Amihud =\frac{1}{N} \sum \frac{|r_t|}{\text{dvol}_t} $$

where $r_t$ is the return on day $t$ and $Volume_t$ is the dollar volume on day $t$. 

The benefit of taking the daily return is to weight the move by change of price and the volume.


In [34]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:42: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=250,min_periods=100,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mea

In [35]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


In [36]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


In [43]:
sector_portfolio_sorts={}
amihud_values={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s],amihud_values[s]=quantile_portfolios_annual_with_rank(sector_zscores[s],cleansed,2)
sector_portfolio_sorts['All'],amihud_values['All']=quantile_portfolios_annual_with_rank(data,cleansed,2)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:224: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:225: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


# Acharya and Pedersen (2005) pricing levels of liquidity and liquidity risk
$$E_t(r_{t+1}^i- r_f) =  \alpha + \kappa E_t(c_{t+1}^i) + \lambda_t(\beta_t + \beta_t^{L1} - \beta_t^{L2} - \beta_t^{L3}) $$
where

* $c_t$:  Liquidity cost 
* $\beta_t^{L1}$:  Commonality in liquidity with the market, $cov(c^i,c^M)$
* $\beta_t^{L2}$:  Return sensitivity to market liquidity, $cov(r^i,c^M)$
* $\beta_t^{L3}$:  Liquidity sensitivity to market returns, $cov(c^i,r^M)$


# Adding expected cost

In [40]:
sector_portfolio_sorts.keys()

dict_keys(['Equities', 'Agriculturals', 'Currencies', 'Fixed Income', 'Metals', 'Energies', 'All'])

In [79]:
final_res={}
for sec in ['Equities', 'Agriculturals', 'Fixed Income', 'Metals', 'Energies', 'All']:
    sector_res=pd.DataFrame()
    for bucket in ['0','1']:
        portfolio_rtn=sector_portfolio_sorts[sec][bucket]
        portfolio_amihud=amihud_values[sec][bucket]
        if sec == 'All':
            market=cleansed.resample(rule='m',how='last').pct_change().median(axis=1).dropna()
            market_amihud=amihud.median(axis=1)
        else:
            market=cleansed[sector_zscores[sec].columns].resample(rule='m',how='last').pct_change().median(axis=1).dropna()
            market_amihud=amihud[sector_zscores[sec].columns].median(axis=1)
        factor=pd.DataFrame()
        factor['Market']=market
        factor['E(c_t)']=portfolio_amihud
        factor['Factor1']=pd.rolling_corr(portfolio_amihud,market_amihud,12,min_periods=12)
        factor['Factor2']=pd.rolling_corr(portfolio_rtn,market_amihud,12,min_periods=12)
        factor['Factor3']=pd.rolling_corr(portfolio_amihud,market,12,min_periods=12)
        ind=portfolio_rtn['2001':].index
        res=sm.OLS(portfolio_rtn.loc[ind],sm.add_constant(factor.loc[ind])).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        #res=sm.OLS(portfolio_rtn.loc[ind],sec.loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        sector_res['Portfolio '+bucket]=res.params
        sector_res['Tstat'+bucket]=res.tvalues
    final_res[sec]=sector_res

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  # This is added back by InteractiveShellApp.init_path()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: pd.rolling_corr is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,min_periods=12).corr(other=<Series>)
  app.launch_new_instance()
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: pd.rolling_corr is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,min_periods=12).corr(other=<Series>)
/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: pd.rolling_corr is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,min_periods=12).c

In [83]:
for sec in ['Agriculturals','Energies','Equities', 'Metals','Fixed Income', 'All']:
    print(sec)
    x=final_res[sec].T
    #x['Sector']=sec
    print(x.round(2)[['Market', 'E(c_t)', 'Factor1', 'Factor2', 'Factor3']].to_latex())

Agriculturals
\begin{tabular}{lrrrrr}
\toprule
{} &  Market &  E(c\_t) &  Factor1 &  Factor2 &  Factor3 \\
\midrule
Portfolio 0 &    0.99 &    0.01 &    -0.00 &    -0.01 &    -0.00 \\
Tstat0      &   28.15 &    1.70 &    -0.12 &    -1.92 &    -0.35 \\
Portfolio 1 &    0.90 &    0.01 &    -0.00 &     0.01 &    -0.00 \\
Tstat1      &   18.95 &    1.62 &    -0.59 &     1.06 &    -0.35 \\
\bottomrule
\end{tabular}

Energies
\begin{tabular}{lrrrrr}
\toprule
{} &  Market &  E(c\_t) &  Factor1 &  Factor2 &  Factor3 \\
\midrule
Portfolio 0 &    1.08 &   -0.00 &    -0.01 &     0.02 &    -0.01 \\
Tstat0      &   17.41 &   -0.10 &    -1.05 &     1.63 &    -1.32 \\
Portfolio 1 &    0.87 &    0.00 &    -0.00 &     0.01 &    -0.01 \\
Tstat1      &   11.86 &    0.12 &    -0.48 &     1.18 &    -0.78 \\
\bottomrule
\end{tabular}

Equities
\begin{tabular}{lrrrrr}
\toprule
{} &  Market &  E(c\_t) &  Factor1 &  Factor2 &  Factor3 \\
\midrule
Portfolio 0 &    0.93 &   -0.01 &    -0.01 &     0.01 &    -0.01